In [ ]:
'https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/33538'
'https://www.kaggle.com/frankherfert/local-validation-with-detailed-product-comparison'
    

load the respective training data  
load the respective test data  
add function(s) specific to that dataset  
train each model on the training data  
pickle the trained model (for safe keeping, not totally necessary)  
make predictions on the test data  
pickle the results  

In [1]:
import os

import pandas as pd
import numpy as np

from predict_submit_funcs import submit_fit_score_pred_log



from collections import OrderedDict

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [2]:
os.chdir('../Data/')

# Without adding department

In [3]:
train_df = pd.read_pickle('full_features_train')
test_df = pd.read_pickle('full_features_test')

## Logistic Regression

In [7]:

def submit_fit_score_pred_log(train_df, test_df):
    """    
    Takes a DataFrame, training, and validation data as its input.
    Returns f1-score, features and their coefficients, and predicted non-re-orders and re-orders.
    """
    test_ids = test_df.loc[:,['product_id','user_id']]
    X_test = test_df.drop(['product_id','user_id'],axis=1) 
    
    X_train = train_df.drop(['product_id','user_id',
                          'latest_cart','in_cart'],axis=1) 
    y_train = train_df['in_cart']
    
    lr = GaussianNB()
    lr.fit(X_train, y_train)
    predictions = pd.DataFrame(lr.predict(X_test))
    
    output = pd.merge(test_ids,predictions,
                      left_index=True,right_index=True)
    
    return output

In [8]:
predictions = submit_fit_score_pred_log(train_df, test_df)

In [13]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4833292 entries, 0 to 4833291
Data columns (total 3 columns):
product_id    int64
user_id       int64
0             int64
dtypes: int64(3)
memory usage: 307.5 MB


## Gaussian Naive Bayes

## Random Forest

# With Polynomial Features

In [ ]:
df = pd.read_pickle('poly_feats_without_dep_df.pkl')

## Gaussian Naive Bayes

# Adding Departments

In [ ]:
df = pd.read_pickle('full_features_with_dep')

## Gaussian Naive Bayes

## Multinomial Naive Bayes

# After Scaling Data

In [ ]:
df = pd.read_pickle('scaled_feats_df')

## Logistic Regression

## Gaussian Naive Bayes

# Scaled Polynomial Features

In [ ]:
df = pd.read_pickle('scaled_poly_feats_without_dep_df.pkl')

## Logistic Regression

## Gaussian Naive Bayes